# 图生图-ControlNet-Canny

同济子豪兄 2023-6-13

MMagic文档：https://github.com/open-mmlab/mmagic/tree/main/configs/controlnet

ControlNet官方Repo：https://github.com/lllyasviel/ControlNet

## 进入 MMagic 主目录

In [1]:
import os
os.chdir('../../mmagic')

In [2]:
# !conda install -c conda-forge ipywidgets -y

## 导入工具包

In [3]:
import cv2
import numpy as np
import mmcv
from mmengine import Config
from PIL import Image

from mmagic.registry import MODELS
from mmagic.utils import register_all_modules

register_all_modules()

/root/miniconda3/envs/mmagic2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip install accelerate
# !conda install xformers -c  xformers -y
!pip install -U xformers

In [5]:
# !python -m xformers.info|

In [6]:
# !pip install -U xformers

## 载入ControlNet模型

In [7]:
cfg = Config.fromfile('configs/controlnet/controlnet-canny.py')
controlnet = MODELS.build(cfg.model).cuda()

06/19 01:42:04 - mmengine - INFO - Set model dtype to 'torch.float32'.
06/19 01:42:07 - mmengine - INFO - Creating runwayml/stable-diffusion-v1-5 by 'HuggingFace'


/mmagic/mmagic/models/archs/wrapper.py:149: FutureWarning: Accessing config attribute `block_out_channels` directly via 'AutoencoderKL' object attribute is deprecated. Please access 'block_out_channels' over 'AutoencoderKL's config object instead, e.g. 'unet.config.block_out_channels'.
  return getattr(self.model, name)


06/19 01:42:08 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'vae'.
06/19 01:42:08 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'unet'.
06/19 01:42:09 - mmengine - INFO - Set model dtype to 'torch.float32'.
06/19 01:42:09 - mmengine - INFO - Enable Xformers for HuggingFace Diffusers' module 'model'.


## 输入Canny边缘图

In [8]:
control_url = 'https://i.pinimg.com/236x/2e/e4/d9/2ee4d97fda423a9cc5bb89211debdb68.jpg'
control_img = mmcv.imread(control_url)
control = cv2.Canny(control_img, 100, 200)
control = control[:, :, None]
control = np.concatenate([control] * 3, axis=2)
control = Image.fromarray(control)

In [9]:
# control

## 咒语Prompt

In [10]:
prompt = 'Room with blue walls and a yellow ceiling.'

## 执行预测

In [11]:
output_dict = controlnet.infer(prompt, control=control)
samples = output_dict['samples']
for idx, sample in enumerate(samples):
    sample.save(f'sample_{idx}.png')
controls = output_dict['controls']
for idx, control in enumerate(controls):
    control.save(f'control_{idx}.png')

/mmagic/mmagic/models/archs/wrapper.py:149: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  return getattr(self.model, name)
  0%|                                                    | 0/20 [00:00<?, ?it/s]/root/miniconda3/envs/mmagic2/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  2.99it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 output_dict = controlnet.infer(prompt, control=control)                                      │
│   2 samples = output_dict['samples']                                                             │
│   3 for idx, sample in enumerate(samples):                                                       │
│   4 │   sample.save(f'sample_{idx}.png')                                                         │
│                                                                                                  │
│ /root/miniconda3/envs/mmagic2/lib/python3.9/site-packages/torch/utils/_contextlib.py:115 in      │
│ decorate_context                                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /mmagic/mmagic/models/editors/controlnet/controlnet.py:537 in infer                              │
│                                                                                                  │
│   534 │   │   │   │   │   noise_pred, t, latents, **extra_step_kwargs)['prev_sample']            │
│   535 │   │                                                                                      │
│   536 │   │   # 8. Post-processing                                                               │
│ ❱ 537 │   │   image = self.decode_latents(latents)                                               │
│   538 │   │                                                                                      │
│   539 │   │   if do_classifier_free_guidance:                                                    │
│   540 │   │   │   controls = torch.split(controls, controls.shape[0] // 2, dim=0)[0]             │
│                                                                                                  │
│ /mmagic/mmagic/models/editors/stable_diffusion/stable_diffusion.py:450 in decode_latents         │
│                                                                                                  │
│   447 │   │   if hasattr(self.vae, 'module'):                                                    │
│   448 │   │   │   image = self.vae.module.decode(latents)['sample']                              │
│   449 │   │   else:                                                                              │
│ ❱ 450 │   │   │   image = self.vae.decode(latents)['sample']                                     │
│   451 │   │   # we always cast to float32 as this does not cause                                 │
│   452 │   │   # significant overhead and is compatible with bfloa16                              │
│   453 │   │   return image.float()                                                               │
│                                                                                                  │
│ /root/miniconda3/envs/mmagic2/lib/python3.9/site-packages/d